In [1]:
import model
import data_set
import generate_sample
import transformers

In [2]:
gpt_path = r".\pretrain_model"
train_path = r".\data\train.json"
test_path = r".\data\test.json"


In [3]:
tk = transformers.BertTokenizer.from_pretrained(gpt_path)

In [4]:
GPT2 = model.GPT2LMHeadModel.from_pretrained(gpt_path)

In [5]:
GPT2.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=21128, bias=False)
)

In [6]:
import torchkeras
import torch
ids = torch.arange(0,32,device="cuda").view(-1,8)

torchkeras.summary(GPT2,ids,batch_size=4)


--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Embedding-1                              [4, 8, 768]           16,226,304
Embedding-2                              [4, 8, 768]              786,432
Dropout-3                                [4, 8, 768]                    0
LayerNorm-4                              [4, 8, 768]                1,536
Conv1D-5                                [4, 8, 2304]            1,771,776
Dropout-6                              [4, 12, 8, 8]                    0
Conv1D-7                                 [4, 8, 768]              590,592
Dropout-8                                [4, 8, 768]                    0
LayerNorm-9                              [4, 8, 768]                1,536
Conv1D-10                               [4, 8, 3072]            2,362,368
NewGELUActivation-11                    [4, 8, 3072]                    0
Conv1D-12                            

'--------------------------------------------------------------------------\nLayer (type)                            Output Shape              Param #\n==========================================================================\nEmbedding-1                              [4, 8, 768]           16,226,304\nEmbedding-2                              [4, 8, 768]              786,432\nDropout-3                                [4, 8, 768]                    0\nLayerNorm-4                              [4, 8, 768]                1,536\nConv1D-5                                [4, 8, 2304]            1,771,776\nDropout-6                              [4, 12, 8, 8]                    0\nConv1D-7                                 [4, 8, 768]              590,592\nDropout-8                                [4, 8, 768]                    0\nLayerNorm-9                              [4, 8, 768]                1,536\nConv1D-10                               [4, 8, 3072]            2,362,368\nNewGELUActivation-11  

In [7]:
res = GPT2.__call__(ids)

In [8]:
trainSet = data_set.GPT2DataSet(
    tokenizer=tk,
    max_len=512,
    title_max_len=64,
    data_dir=".\data",
    data_set_name="train",
    path_file=".\data\\train.json"    
)

In [9]:
torch.cuda.empty_cache()
torch.cuda.memory_usage()

0

In [10]:
samples = [trainSet[i] for i in range(10,11)]
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
batch = data_set.collate_func(samples)
print(tk.decode(samples[0]["input_ids"]))
res = GPT2.forward(
    input_ids=batch["input_ids"].cuda(),
    labels=batch["input_ids"].cuda(),
    mask=batch["mask"].cuda()
)

input_ids = batch["input_ids"].cuda()
mask      = batch["mask"].cuda()
label     = batch["input_ids"].cuda()
pkvs      = None
import time
t1 = time.time()
torch.cuda.synchronize()
for i in range(10):
    GPTOUT= GPT2.forward(
        input_ids=input_ids,
        mask = mask,
        past_key_values=pkvs
    )
    last_states,pkvs = GPTOUT
    logits = last_states[...,-1,:]
    next_token_ids = logits.argmax(-1).unsqueeze(1)
    token = tk.decode([next_token_ids.item()])
    print(token)
    input_ids = torch.concat([input_ids,next_token_ids],dim=-1)

print(f"time usage:{time.time()-t1}")
    
    


[CLS] 随 着 信 息 技 术 的 发 展, 微 博 正 在 被 广 泛 地 运 用 到 教 育 领 域. 微 博 教 育 是 否 适 用 于 高 校 公 共 艺 术 教 育 呢 ？ 文 章 以 高 校 公 共 艺 术 教 育 微 博 教 育 为 研 究 对 象, 通 过 对 公 共 艺 术 课 程 微 博 教 育 应 用 现 状 的 调 查, 从 基 于 微 博 视 域 的 国 内 外 教 育 应 用 研 究 现 状 、 微 博 视 域 下 艺 术 学 科 的 微 博 教 育 现 状 、 高 校 公 共 艺 术 教 育 课 程 应 用 微 博 教 学 的 可 行 性 分 析 、 微 博 视 角 下 高 校 公 共 艺 术 教 育 课 程 教 学 模 式 设 计 等 四 个 方 面 对 微 博 在 高 校 公 共 艺 术 教 育 中 的 可 行 性 进 行 分 析. [SEP] 高 校 公 共 艺 术 教 育 微 博 教 育 的 可 行 性 研 究 [SEP]
微
博
教
育


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
next_token_ids.item()